In [2]:
# -*- coding: utf-8 -*-
import re
import time
from urllib import request

import mysql.connector
from bs4 import BeautifulSoup

# 网页url前缀
url_prefix = "http://bio.nifdc.org.cn/pqf/"
# 首页url
base_url = "http://bio.nifdc.org.cn/pqf/search.do?formAction=pqfGsByJG&orgId=1"

In [ ]:
class Collector(object):
    def __init__(self):
        self.links = []

    # 搜集所有URLs
    def collect_all_urls(self):
        req_url = base_url
        req = request.Request(req_url)
        req.add_header('User-agent',
                       'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                       'Chrome/67.0.3396.99 '
                       'Safari/537.36')

        # 读取网页
        response = request.urlopen(req)
        the_page = response.read()

        # 解析网页
        page = BeautifulSoup(the_page, "html.parser")

        anchors = page.find_all(name="a")
        for anchor in anchors:
            self.links.append(anchor['href'])

    # 根据collect_all_urls()搜集的urls查询数据
    def query(self):
        num = 0

        # 在这里修改成你的数据库连接信息
        conn = mysql.connector.connect(user='root', password='123456', host='127.0.0.1', database='yimiao')
        cursor = conn.cursor()

        for parturl in self.links:
            req_url = url_prefix + parturl
            req = request.Request(req_url)
            req.add_header('User-agent',
                           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                           'Chrome/67.0.3396.99 '
                           'Safari/537.36')

            # 读取网页
            response = request.urlopen(req)
            the_page = response.read()

            # 解析网页
            page = BeautifulSoup(the_page, "html.parser")

            table = page.find(name="table", attrs={"class": re.compile("list_table")})

            num += 1
            print("Visit link No.%i: " % num, req_url)

            rows = table.find_all(name="tr")

            print("Start writing link No.%i: " % num, req_url)

            for row in rows[1:]:
                items = row.find_all(name="td")
                itemlist = []
                for item in items:
                    itemlist.append(item.text)
                cursor.execute(
                    'insert into yimiao_info (product_name, spec, batch_no, issue_or_not_volume, expiration_date, '
                    'manufacturer, '
                    'checking_no, certificate_no, report_no, issue_date, issue_conclusion, issue_authority)'
                    ' values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', itemlist[1:13])

            conn.commit()
            print("Fishing writing link No.%i: " % num, req_url)

        cursor.close()




In [8]:
if __name__ == "__main__":
    start_time = time.time()

    collector = Collector()
    collector.collect_all_urls()

    num = 1
    for link in collector.links:
        print(num, '. ', link)
        num = num + 1

    collector.query()

    finish_time = time.time() - start_time

    print('Used time(s):' + repr(finish_time))

    ÐòºÅ                               ²úÆ·Ãû³Æ                                               ¹æ¸ñ                ÅúºÅ Ç©·¢Á¿/¾ÜÇ©Á¿     ÓÐÐ§ÆÚÖÁ                   Éú²úÆóÒµ         ÊÕ¼ì±àºÅ         Ö¤Êé±àºÅ     ±¨¸æ±àºÅ     Ç©·¢ÈÕÆÚ Ç©·¢½áÂÛ   ÅúÇ©·¢»ú¹¹
0      1  SabinÖê¼¹Ëè»ÒÖÊÑ×Ãð»îÒßÃç£¨VeroÏ¸°û£©  Ã¿Æ¿0.5ml£¬Ã¿1´ÎÈËÓÃ¼ÁÁ¿Îª0.5ml¡£º¬¼¹Ëè»ÒÖÊÑ×²...           202002005       98475Æ¿   2022年2月12日            北京生物制品研究所有限责任公司  SP0404202003347     批签中检20201664            /  2020年05月07日     予以签发  中国食品药品检定研究院
1      2                Sabin株脊髓灰质炎灭活疫苗（Vero细胞）  每瓶0.5ml，每1次人用剂量为0.5ml。含脊髓灰质炎病毒抗原量Ⅰ型15DU、Ⅱ型45DU...           202002002        96615瓶    2022年2月9日            北京生物制品研究所有限责任公司  SP0404202003344     批签中检20201661            /  2020年05月07日     予以签发  中国食品药品检定研究院
2      3                            冻干甲型肝炎减毒活疫苗  复溶后0.5ml/瓶。每1次人用剂量为0.5ml，含甲型肝炎活病毒应不低于6.501gCCI...            20191248       106705瓶   2021年6月14日            长春生物制品研究所有限责任公司  SP0403202002575     批签中检20201693            /  2020年05月07日   